In [ ]:
"""
CARMEN Calcium Multiplexing Post-Processing Script
--------------------------------------------------

This script combines and background-subtracts ROI intensity measurements from four-channel calcium imaging
experiments processed using the CARMEN Fiji macro (CARMEN_MeasureFourChannelROI.ijm).
The script assumes that the last ROI in each CSV file represents a background region and uses its values for background subtraction. 
It expects CSV files to be exported per image and per channel (e.g., "Image01_Intensity FP1.csv" through "Image01_Intensity FP4.csv"), 
and merges these into a single Excel file containing both the original measurements and background-corrected data.

Functionality:
- Scans a folder for CSV files matching the expected naming convention.
- Groups CSVs by image base name, ensuring exactly four files per group (FP1 to FP4).
- Loads the files and writes the raw data into separate Excel sheets (FP1, FP2, FP3, FP4).
- Applies background subtraction using the last column (assumed to be background values).
- Saves corrected data in additional sheets (FP1_bs, FP2_bs, FP3_bs, FP4_bs) within the same Excel file.

Assumptions:
- CSV files follow the naming convention: "<base_name>_Intensity FPX.csv"
- The last column in each CSV contains background intensity values.
- All CSVs are located in a single folder, defined by `file_path`.

Outputs:
- For each image (base_name), creates an Excel file: "<base_name>_analysis.xlsx"
  - Sheets: FP1, FP2, FP3, FP4          — Raw data
  - Sheets: FP1_bs, FP2_bs, FP3_bs, FP4_bs — Background-subtracted data

Usage:
- Set the `file_path` to the folder containing CSV exports from the CARMEN Fiji macro.
- Run the script in a Python environment with `pandas` installed.
"""

import pandas as pd
import os
from collections import defaultdict

# Define the path to your CSV files
file_path = r"your/file/path"  # Replace with the actual path to your files

# Set the suffix for the output Excel file
excel_suffix = '_analysis'

# Dictionary to hold the grouped CSV files by image base name
grouped_files = defaultdict(list)

# Group CSV files by image base name (e.g., Image01 from Image01_Intensity FP1.csv)
for file in os.listdir(file_path):
    if file.endswith(".csv"):
         # Get the part of the filename before "_FP1", "_FP2", etc.
        base_name = file.rsplit('_', 1)[0]
        grouped_files[base_name].append(file)

# List to hold paths of all created Excel files
excel_file_paths = []

# Process each group of CSV files
for base_name, files in grouped_files.items():
    # Only process groups that contain exactly four files (one per channel)
    if len(files) == 4:
        # Ensure files are in the correct order: FP1, FP2, FP3, FP4
        files.sort(key=lambda x: int(x[-5]))

        # Dictionary to store the loaded CSV data
        csv_files = {}
        
        # Load each CSV file into a dictionary, using its suffix (FP1, FP2, etc.) as the key
        for file in files:
            suffix = file.rsplit('_', 1)[1].split('.')[0]  # Extract FB1, FB2, etc.
            full_path = os.path.join(file_path, file)
            csv_files[suffix] = pd.read_csv(full_path)

        # Create an Excel file to hold both raw and background-subtracted data
        output_excel_path = os.path.join(file_path, f'{base_name}{excel_suffix}.xlsx')
        
       
        # Write data to excel file
        with pd.ExcelWriter(output_excel_path) as writer:
            for sheet_name, df in csv_files.items():
                # Write original (raw) data to separate sheets
                df.to_excel(writer, sheet_name=sheet_name, index=False)

                # Identify the last column, assumed to be the background ROI
                background_col = df.columns[-1]
                background_values = df[background_col]

                # Subtract background from all other numeric columns
                corrected_df = df.copy()
                for col in df.columns[:-1]:
                    if pd.api.types.is_numeric_dtype(df[col]):
                        corrected_df[col] = df[col] - background_values

                # Remove the background column from the output
                corrected_df = corrected_df.drop(columns=background_col)

                # Save background-subtracted data in a new sheet with "_bs" suffix
                corrected_sheet_name = f"{sheet_name}_bs"
                corrected_df.to_excel(writer, sheet_name=corrected_sheet_name, index=False)

        # Confirm export
        print(f"CSV files for {base_name} successfully saved to {base_name}{excel_suffix}.xlsx.")
    else:
        # Skip incomplete groups and notify the user
        print(f"Warning: {base_name} does not have exactly 4 corresponding CSV files.")
